In [1]:
from pyspark.sql import SQLContext, SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType, StructType, StructField
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorAssembler
#sc.install_pypi_package("pandas")
#sc.install_pypi_package("s3fs")
import pandas as pd

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1591935046141_0006,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/a4/5f/1b6e0efab4bfb738478919d40b0e3e1a06e3d9996da45eb62a77e9a090d9/pandas-1.0.4-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/b8/e4/b8fc59248399d2482b39340ec9be4bb2493846ac23641b43115a7e5cd675/s3fs-0.4.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/0f/31/f27a81686b2f1b2f6776bd5db10efc7d88f28a50e8888f55409ef6501a50/fsspec-0.7.4-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e1/e5/df302e8017440f111c11cc41a6b432838672f5a70aa29227bf58149dc72f/urllib3-1.25.9-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/22/cd/a6aa959dca619918ccb55023b4cb151949c64d4d5d55b3f4ffd7eee0c6e8/docutils-0.15.2-py3-none-any.whl

In [2]:
#sc = SparkContext()
sqlcontext = SQLContext(sc)
sc.setLogLevel('ERROR')

path = 's3://nu-zach-s3/trading.csv'
trading = sqlcontext.read.csv(path, header = True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
trading.filter((col('trade_id')==1)).select('time_stamp','bar_num','profit','trade_id').show(130)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------+------+--------+
|         time_stamp|bar_num|profit|trade_id|
+-------------------+-------+------+--------+
|2015-08-03 11:15:00|    120|   -57|       1|
|2015-08-03 11:14:00|    119|   -40|       1|
|2015-08-03 11:13:00|    118|   -29|       1|
|2015-08-03 11:12:00|    117|   -70|       1|
|2015-08-03 11:11:00|    116|   -67|       1|
|2015-08-03 11:10:00|    115|   -35|       1|
|2015-08-03 11:09:00|    114|   -26|       1|
|2015-08-03 11:08:00|    113|   -47|       1|
|2015-08-03 11:07:00|    112|   -47|       1|
|2015-08-03 11:06:00|    111|   -48|       1|
|2015-08-03 11:05:00|    110|   -48|       1|
|2015-08-03 11:04:00|    109|   -39|       1|
|2015-08-03 11:03:00|    108|   -17|       1|
|2015-08-03 11:02:00|    107|   -26|       1|
|2015-08-03 11:01:00|    106|    -7|       1|
|2015-08-03 11:00:00|    105|    17|       1|
|2015-08-03 10:59:00|    104|    12|       1|
|2015-08-03 10:58:00|    103|    12|       1|
|2015-08-03 10:57:00|    102|    5

In [3]:
# clean date and create groups for bar_num
trading_clean = trading.orderBy("time_stamp",ascending=False)\
                       .withColumn('date', to_date('time_stamp','yyyy-MM-dd'))\
                       .withColumn('year',year(col('date')))\
                       .withColumn('Month', month(col('date')))\
                       .withColumn('group', ceil(col('bar_num')/10))\
                       .drop('time_stamp')\
                       .drop('direction')\
                       .filter(col('year')>=2007)\
                       .filter((col('year')<=2015) & (col('month')<=7))\
                        
# Change string type to integer
trading_clean = trading_clean.select([col(c).cast("integer") for c in trading_clean.columns])


# create new features
group_data = trading_clean.groupBy('trade_id','group')
group_data = group_data.agg(min(col('profit')).alias('profit_min'),
                            max(col('profit')).alias('profit_max'),
                            avg(col('profit')).alias('profit_avg'),
                            sum(col('profit')).alias('profit_sum'))

trading_clean = trading_clean.join(group_data, on = ['trade_id', 'group'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
trading_clean.filter((col('trade_id')==1) & (col('group')==9)).select('bar_num','profit','trade_id','year','month','group','profit_sum').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+--------+----+-----+-----+----------+
|bar_num|profit|trade_id|year|month|group|profit_sum|
+-------+------+--------+----+-----+-----+----------+
|     81|   119|       1|2015|    8|    9|      1226|
|     82|   134|       1|2015|    8|    9|      1226|
|     83|   144|       1|2015|    8|    9|      1226|
|     84|   132|       1|2015|    8|    9|      1226|
|     85|   122|       1|2015|    8|    9|      1226|
|     86|   108|       1|2015|    8|    9|      1226|
|     87|   114|       1|2015|    8|    9|      1226|
|     88|   123|       1|2015|    8|    9|      1226|
|     89|   122|       1|2015|    8|    9|      1226|
|     90|   108|       1|2015|    8|    9|      1226|
+-------+------+--------+----+-----+-----+----------+

In [18]:
trading_clean.sort('trade_id','group').select('bar_num','profit','trade_id','year','month','group','profit_sum').show(130)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+--------+----+-----+-----+----------+
|bar_num|profit|trade_id|year|month|group|profit_sum|
+-------+------+--------+----+-----+-----+----------+
|      1|    13|       1|2015|    8|    1|      -229|
|      4|    18|       1|2015|    8|    1|      -229|
|      7|   -60|       1|2015|    8|    1|      -229|
|      8|   -62|       1|2015|    8|    1|      -229|
|      2|     1|       1|2015|    8|    1|      -229|
|      6|   -30|       1|2015|    8|    1|      -229|
|     10|   -39|       1|2015|    8|    1|      -229|
|      9|   -71|       1|2015|    8|    1|      -229|
|      5|     9|       1|2015|    8|    1|      -229|
|      3|    -8|       1|2015|    8|    1|      -229|
|     11|   -41|       1|2015|    8|    2|      -238|
|     18|   -19|       1|2015|    8|    2|      -238|
|     20|    20|       1|2015|    8|    2|      -238|
|     16|   -37|       1|2015|    8|    2|      -238|
|     12|   -30|       1|2015|    8|    2|      -238|
|     15|   -11|       1|201

In [5]:
trading_clean_subset = trading_clean.select(col('trade_id'),
                                          col('group'),
                                          col('profit_min'),
                                          col('profit_max'),
                                          col('profit_avg'),
                                          col('profit_sum'))\
                                    .groupBy('trade_id','group','profit_min','profit_max','profit_avg','profit_sum').count()\
                                    .drop('count').sort('trade_id','group')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
trading_clean_subset.select('trade_id','group','profit_min').show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----+----------+
|trade_id|group|profit_min|
+--------+-----+----------+
|       1|    1|       -71|
|       1|    2|       -45|
|       1|    3|        20|
|       1|    4|        25|
|       1|    5|       -29|
|       1|    6|        -6|
|       1|    7|        40|
|       1|    8|        51|
|       1|    9|       108|
|       1|   10|        44|
|       1|   11|       -48|
|       1|   12|       -70|
|       2|    1|       -52|
|       2|    2|       -35|
|       2|    3|        24|
|       2|    4|        75|
|       2|    5|        58|
|       2|    6|        30|
|       2|    7|        37|
|       2|    8|        59|
|       2|    9|        57|
|       2|   10|        97|
|       2|   11|       150|
|       2|   12|       146|
|       3|    1|       -36|
|       3|    2|        46|
|       3|    3|        11|
|       3|    4|         4|
|       3|    5|       -13|
|       3|    6|       -44|
|       3|    7|       -30|
|       3|    8|       -50|
|       3|    9|    

In [29]:
trading_clean_subset.filter((col('trade_id')==1) & (col('group')==9)).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----+----------+----------+----------+----------+
|trade_id|group|profit_min|profit_max|profit_avg|profit_sum|
+--------+-----+----------+----------+----------+----------+
|       1|    9|       108|       144|     122.6|      1226|
+--------+-----+----------+----------+----------+----------+

In [30]:
# for each group, not only var12-var78 will be used as features, 
# but the previous groups' profit(min,max,mean) as additional features
#Here we are defining the schema of an empty dataframe that will grow as we traverse out loop
schema = StructType([StructField('trade_id', IntegerType(), False), 
                     StructField(('group'), IntegerType(), False), 
                     StructField(('lag_profit_min'), IntegerType(), False),
                     StructField(('lag_profit_max'), IntegerType(), False),
                     StructField(('lag_profit_avg'), IntegerType(), False),
                     StructField(('lag_profit_sum'), IntegerType(), False)])

results = sqlcontext.createDataFrame([], schema)
                     
for group in range(1,13):      
    if group == 1:
        lag_profit = trading_clean_subset.filter(col('group')==1)\
                                         .withColumn('lag_profit_min',col('profit_min'))\
                                         .withColumn('lag_profit_max',col('profit_max'))\
                                         .withColumn('lag_profit_avg',col('profit_avg'))\
                                         .withColumn('lag_profit_sum',col('profit_sum'))\
                                         .select(col('trade_id'),
                                               col('group'),
                                               col('lag_profit_min'),
                                               col('lag_profit_max'),
                                               col('lag_profit_avg'),
                                               col('lag_profit_sum'))
    else:
        previous_level = trading_clean_subset.filter(col('group')==group-1)\
                                  .select(col('trade_id'),
                                          col('group'),
                                          col('profit_min'),
                                          col('profit_max'),
                                          col('profit_avg'),
                                          col('profit_sum'))
        
        lag_profit = previous_level.withColumn('group',col('group')+1)\
                                   .select(col('trade_id'),
                                           col('group'),
                                           col('profit_min').alias('lag_profit_min'),
                                           col('profit_max').alias('lag_profit_max'),
                                           col('profit_avg').alias('lag_profit_avg'),
                                           col('profit_sum').alias('lag_profit_sum'))

    results = results.union(lag_profit)

# join with trading_clean to create the frame for modelling
trading_model = trading_clean.join(results, on = ['trade_id','group'], how='inner')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
trading_model.select('bar_num','profit','trade_id','year','month','group','profit_sum','lag_profit_sum').show(120)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+--------+----+-----+-----+----------+--------------+
|bar_num|profit|trade_id|year|month|group|profit_sum|lag_profit_sum|
+-------+------+--------+----+-----+-----+----------+--------------+
|     90|   108|       1|2015|    8|    9|      1226|           789|
|     89|   122|       1|2015|    8|    9|      1226|           789|
|     88|   123|       1|2015|    8|    9|      1226|           789|
|     87|   114|       1|2015|    8|    9|      1226|           789|
|     86|   108|       1|2015|    8|    9|      1226|           789|
|     85|   122|       1|2015|    8|    9|      1226|           789|
|     84|   132|       1|2015|    8|    9|      1226|           789|
|     83|   144|       1|2015|    8|    9|      1226|           789|
|     82|   134|       1|2015|    8|    9|      1226|           789|
|     81|   119|       1|2015|    8|    9|      1226|           789|
|     10|    47|       3|2015|    8|    1|       155|           155|
|      9|    36|       3|2015|    

In [32]:
trading_model.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- trade_id: integer (nullable = true)
 |-- group: integer (nullable = true)
 |-- bar_num: integer (nullable = true)
 |-- profit: integer (nullable = true)
 |-- var12: integer (nullable = true)
 |-- var13: integer (nullable = true)
 |-- var14: integer (nullable = true)
 |-- var15: integer (nullable = true)
 |-- var16: integer (nullable = true)
 |-- var17: integer (nullable = true)
 |-- var18: integer (nullable = true)
 |-- var23: integer (nullable = true)
 |-- var24: integer (nullable = true)
 |-- var25: integer (nullable = true)
 |-- var26: integer (nullable = true)
 |-- var27: integer (nullable = true)
 |-- var28: integer (nullable = true)
 |-- var34: integer (nullable = true)
 |-- var35: integer (nullable = true)
 |-- var36: integer (nullable = true)
 |-- var37: integer (nullable = true)
 |-- var38: integer (nullable = true)
 |-- var45: integer (nullable = true)
 |-- var46: integer (nullable = true)
 |-- var47: integer (nullable = true)
 |-- var48: integer (nullable = true)
 

In [33]:
results.filter((col('trade_id')==1) & (col('group')==9)).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-33:
Traceback (most recent call last):
  File "/emr/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/emr/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 2579



+--------+-----+--------------+--------------+--------------+--------------+
|trade_id|group|lag_profit_min|lag_profit_max|lag_profit_avg|lag_profit_sum|
+--------+-----+--------------+--------------+--------------+--------------+
|       1|    9|            51|           100|          78.9|           789|
+--------+-----+--------------+--------------+--------------+--------------+

In [13]:
trading_model.where(col('bar_num').isNull()).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [34]:
############ modelling #############
# lists to store mape
mape_list = []

# features
fe = ['bar_num','var12', 'var13','var14', 'var15', 'var16', 'var17', 'var18', 'var23', 'var24', 'var25',
       'var26', 'var27', 'var28', 'var34', 'var35', 'var36', 'var37', 'var38','var45', 'var46', 'var47', 
       'var48', 'var56', 'var57', 'var58', 'var67','var68', 'var78',
      'lag_profit_min','lag_profit_max','lag_profit_avg','lag_profit_sum']


assembler = VectorAssembler(inputCols=fe, outputCol='features').setHandleInvalid("skip")
rf = RandomForestRegressor(labelCol="profit", featuresCol="features")
pipeline = Pipeline(stages=[assembler, rf])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
# train model
for yr in range(2008,2016):
    # train from Jan to June, or July to Dec
    for mo in (1,7):
        if (yr == 2015) & (mo==7): break
            
        train = trading_model.filter((col('year')==yr) & (col('month') >= mo) & (col('month') <= mo+5))
        test = trading_model.filter((col('year')== (yr + mo//7)) & (col('month')== (mo+6)%12))

        model_rf = pipeline.fit(train)
        test_rf = model_rf.transform(test)
        
        # calculating mape
        mape = test_rf.select('profit','prediction')\
                      .withColumn("ape",abs((col('prediction')-col('profit'))/col('profit')))\
                      .agg(avg(col('ape'))).collect()[0][0]
                                           
        date = str(yr) + '-' + str((mo+6)%12)
        mape_list.append([date,mape])  

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-35:
Traceback (most recent call last):
  File "/emr/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/emr/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 3478



In [36]:
# Transform to dataframe
df = pd.DataFrame(mape_list, columns=['Date','MAPE'])
mmm_list=[['Max MAPE:', df.MAPE.max()],['Min MAPE:',df.MAPE.min()],['Average MAPE:',df.MAPE.mean()]]
df1 = pd.DataFrame(mmm_list, columns=['Date','MAPE'])
df_final = pd.concat([df,df1],axis=0)         
df_final.to_csv('s3://nu-zach-s3/Exercise3-3.txt',index=False, sep=' ', mode='a')
df_final.to_csv('s3://nu-zach-s3/Exercise3-3.csv',index=False)
# sc.stop() 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…